In [1]:
import os
os.chdir('/home/149/ab8992/cosima_regional/development/regional-mom6')
# from recipes import surfacespeed
from importlib import reload
# from ttidelib import logmsg
import shutil
import subprocess
import dask
import xarray as xr
from matplotlib import pyplot as plt
from dask.distributed import Client, default_client
from pathlib import Path
import sys
# sys.path.append('/home/149/ab8992/cosima_regional/development/regional-mom6/regional_mom6')
import regional_mom6 as rm
try:
    # Try to get the existing Dask client
    client = default_client()
except ValueError:
    # If there's no existing client, create a new one
    client = Client()

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 112.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40795,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 112.00 GiB
Comm: tcp://127.0.0.1:43383,Total threads: 4
Dashboard: /proxy/38489/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:42833,


In [3]:
os.chdir('/home/149/ab8992/tasman-tides')

import ttidelib as tt
import recipes as recipes


In [4]:
     

data = tt.collect_data("full-20",rawdata = ["u","v","ahh"],timerange=(0,24 * 365 * 9))
data

loading u...	done.
loading v...	done.
loading ahh...	

2024-02-21 20:10:14,150 - tornado.application - ERROR - Uncaught exception GET /status/ws (::1)
HTTPServerRequest(protocol='http', host='localhost:8888', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/tornado/websocket.py", line 937, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/tornado/web.py", line 3290, in wrapper
    return method(self, *args, **kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


done.
0.3.0


<xarray.Dataset>
Dimensions:  (zl: 100, time: 78840, xb: 376, yb: 101)
Coordinates:
  * zl       (zl) float64 2.704 8.126 13.58 ... 5.236e+03 5.339e+03 5.443e+03
  * time     (time) float64 1.0 2.0 3.0 4.0 ... 7.884e+04 7.884e+04 7.884e+04
  * xb       (xb) float64 -0.0 4.0 8.0 12.0 ... 1.492e+03 1.496e+03 1.5e+03
  * yb       (yb) float64 -200.0 -196.0 -192.0 -188.0 ... 192.0 196.0 200.0
    lon      (yb, xb) float64 dask.array<chunksize=(101, 376), meta=np.ndarray>
    lat      (yb, xb) float64 dask.array<chunksize=(101, 376), meta=np.ndarray>
Data variables:
    u        (time, zl, yb, xb) float32 dask.array<chunksize=(308, 84, 4, 321), meta=np.ndarray>
    v        (time, zl, yb, xb) float32 dask.array<chunksize=(308, 84, 4, 321), meta=np.ndarray>
    ahh      (time, zl, yb, xb) float32 dask.array<chunksize=(308, 84, 4, 321), meta=np.ndarray>
    bathy    (yb, xb) float64 dask.array<chunksize=(101, 376), meta=np.ndarray>
Attributes:
    _FillValue:   nan
    long_name:    Velocity along beam (Eastward positive)
    coordinates:  lat lon

array(2.)

2024-02-21 18:00:03,579 - distributed.nanny - WARNING - Worker process still alive after 3.1999983215332035 seconds, killing


In [3]:
recipes.save_filtered_vels("full-20","output*")

Startdask: found existing client
<Client: 'tcp://127.0.0.1:46569' processes=7 threads=28, memory=112.00 GiB>


2024-02-21 15:49:36,579 - tornado.application - ERROR - Exception in callback functools.partial(<function TCPServer._handle_connection.<locals>.<lambda> at 0x1516923e3760>, <Task finished name='Task-202898' coro=<BaseTCPListener._handle_stream() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/comm/tcp.py:655> exception=MemoryError((7022364365220496711,), dtype('uint8'))>)
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/tornado/ioloop.py", line 738, in _run_callback
    ret = callback()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/tornado/tcpserver.py", line 387, in <lambda>
    gen.convert_yielded(future), lambda f: f.result()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/comm/tcp.py", line 667, in _handle_stream
    await self.on_connec

CancelledError: ('all-aggregate-4d89cfece1182dc70ff389c18f50f2aa',)

In [ ]:
def save_filtered_vels(experiment,outputs,recompute = False):
    """
    Calculate the filtered velocities over 149 hours and save u'u', v'v', u'v' all averaged over 149 hours as separate files
    """
    # startdask()
    basepath = gdata / "postprocessed" / experiment

    m2 = 360 / 28.984104 ## Period of m2 in hours
    averaging_window = int(12 * m2) ## this comes out to be 149.0472 hours, so close enough to a multiple of tidal periods
    m2f = 1/ m2    ## Frequency of m2 in radians per hour

    data = tt.collect_data(
        experiment,
        outputs=outputs,
        rawdata = ["u","v","ahh"],
        # chunks = {"xb":-1,"zl":10}
        )
    print("Data loaded")
    for i in range(0,len(data.time) // averaging_window):
        mid_time =  data.time[int(np.floor((i + 0.5) * averaging_window)) ] ## Middle of time window time

        ## Here skip the time slice if it already exists and recompute is False
        if os.path.exists(basepath / "UU" / "topdown" / f"UU_time-{int(mid_time.values)}.nc") and not recompute:
            print("Skipping time",f"{i} = {mid_time}")
            continue

        print("Processing time slice",f"{i} = {mid_time}")
        u_ = data.u.isel(
                time = slice(i * averaging_window, (i + 1) * averaging_window)
                ).chunk({"time":-1}).drop(["lat","lon"]).fillna(0)
        v_ = data.v.isel(
                time = slice(i * averaging_window, (i + 1) * averaging_window)
                ).chunk({"time":-1}).drop(["lat","lon"]).fillna(0)
        ahh = data.ahh.isel(
                time = slice(i * averaging_window, (i + 1) * averaging_window)
                ).chunk({"time":-1}).drop(["lat","lon"]).fillna(0)
        U = tt.m2filter(
            u_,
            m2f).persist()
        V = tt.m2filter(
            v_,
            m2f).persist()
        
        # Calculate dissipation as viscosity * laplacian(u)^2 
        laplacian2 = (U.differentiate("xb").differentiate("xb") + V.differentiate("yb").differentiate("yb")
                )**2
        dissipation = (laplacian2.mean("time") * ahh.mean("time")).expand_dims("time").assign_coords(time = [mid_time]).rename("dissipation")

        data_to_save = {
            "UU" : (U * U).mean("time").expand_dims("time").assign_coords(time = [mid_time]).rename("UU"),
            "VV" : (V * V).mean("time").expand_dims("time").assign_coords(time = [mid_time]).rename("VV"),
            "UV" : (U * V).mean("time").expand_dims("time").assign_coords(time = [mid_time]).rename("UV"),
            "dissipation" : dissipation
        }

        for key in data_to_save:
            tt.save_ppdata(
                data_to_save[key].sel(yb = 0,method = "nearest"),
                data_to_save[key].integrate("zl"),
                basepath / key,
                recompute=recompute
            )

    return 

In [ ]:
/home/157/ahg157/repos/mom5/src/tools/check_mask/check_mask --grid_file ocean_mosaic.nc --ocean_topog topog.nc --layout 80,80 --halo 4

In [ ]:
test1 = xr.open_dataset("/g/data/nm03/ab8992/outputs/full-20/output100/u/u_y01.nc")["u"]
test2 = xr.open_dataset("/g/data/nm03/ab8992/outputs/full-20/output101/u/u_y01.nc")["u"]
a = "1"
for i in [test1,test2]:
    i.isel(zl = 0,xb = slice(0,6)).to_netcdf(f"/g/data/v45/ab8992/dropbox/test_file{a}.nc")
    a = "2"

In [3]:
test1

<xarray.DataArray 'u' (time: 360, zl: 100, yb: 6, xb: 376)>
[81216000 values with dtype=float32]
Coordinates:
  * zl       (zl) float64 2.704 8.126 13.58 ... 5.236e+03 5.339e+03 5.443e+03
  * time     (time) object 2014-02-09 01:00:00 ... 2014-02-24 00:00:00
  * xb       (xb) float64 -0.0 4.0 8.0 12.0 ... 1.492e+03 1.496e+03 1.5e+03
  * yb       (yb) float64 -176.0 -172.0 -168.0 -164.0 -160.0 -156.0
    lon      (yb, xb) float64 ...
    lat      (yb, xb) float64 ...
Attributes:
    long_name:  Velocity along beam (Eastward positive)